In [1]:
%pip install llama-index-llms-mistralai
!pip install llama-index 

In [2]:
from llama_index.llms.mistralai import MistralAI
import os 

llm = MistralAI(api_key=os.environ['MISTRAL_API_KEY'])

In [3]:
llm.complete("Hello, how are you?")

In [4]:
!pip install datasets

In [5]:
from datasets import load_dataset
from pathlib import Path

try:
    dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

try:
    Path("data").mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"Error creating directory: {e}")
    raise

for i, persona in enumerate(dataset):
    try:
        with open(Path("data") / f"persona_{i}.txt", "w", encoding="utf-8") as f:
            f.write(persona["persona"])
    except Exception as e:
        print(f"Error writing file persona_{i}.txt: {e}")
        raise

In [6]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()

In [7]:
pip install llama-index-embeddings-mistralai


In [8]:
from llama_index.core import Document
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# pipeline = IngestionPipeline(
#     transformations=[
#         SentenceSplitter(chunk_overlap=0),
#         MistralAIEmbedding(),
#     ]
# )

# nodes = await pipeline.arun(documents=documents)

##Never ends


pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_overlap=0),
        MistralAIEmbedding(),
    ]
)

nodes = await pipeline.arun(documents=[Document.example()])

In [9]:
nodes

In [10]:
print(Document.example())

In [11]:
## Tutorial data
!pip install llama-index-vector-stores-chroma

In [12]:
## Run in 17 minutes

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
import time

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=50, chunk_overlap=0),
        MistralAIEmbedding(),
    ],
    vector_store=vector_store,
)

batch_size = 10
all_nodes = []



for i in range(0, 10, batch_size):      # for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    nodes = await pipeline.arun(documents=documents[:10])
    all_nodes.extend(nodes)
    print(f"Processed batch {i // batch_size + 1}, total nodes: {len(all_nodes)}")
    time.sleep(2)


In [13]:
from llama_index.core import VectorStoreIndex

embed_model = MistralAIEmbedding()
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [14]:
nodes

In [15]:
llm = MistralAI(model='codestral-latest')

In [16]:
print(f"Number of nodes in index: {len(index.docstore.docs)}")

In [17]:
!pip install nest_asyncio

In [22]:
import nest_asyncio

nest_asyncio.apply()
llm = MistralAI(model='codestral-latest')
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)

response = query_engine.query("Respond using a persona that describes author and travel experiences?")
print(response)

In [20]:
response = query_engine.query("Respond using a persona that describes author and travel experiences?"
)
print(response)

In [21]:
response = query_engine.query("What travel experiences are mentioned in the documents?")
print(response)

In [23]:
query_engine.query("What is the meaning of life?")


In [ ]:
# Pour adapter a des données tabulaires, il faut faire un splitter par ligne en utilisant iterrows() de pandas

# Problème car c'est très long 
# A essayer



# import asyncio
# from tqdm.asyncio import tqdm_asyncio

# async def process_all_rows(documents, batch_size=100):
#     tasks = []
    
#     # Create batch processing tasks
#     for i in range(0, len(documents), batch_size):
#         batch = documents[i:i+batch_size]
#         tasks.append(pipeline.arun(documents=batch))
    
#     # Process all batches concurrently
#     all_results = await tqdm_asyncio.gather(*tasks)
    
#     # Flatten results
#     all_nodes = []
#     for result in all_results:
#         all_nodes.extend(result)
    
#     return all_nodes

# # Run the async function
# nodes = await process_all_rows(documents)

In [27]:
from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query("What battles took place in New York City in the American Revolution?")

eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

In [28]:
print(response)

In [30]:
print(eval_result)

In [35]:
!pip install -U llama-index-callbacks-arize-phoenix
# pip install -q arize-phoenix

In [38]:
!pip install -q arize-phoenix

In [41]:
from phoenix.otel import register 

tracer_provider = register(
    project_name="RAG with llama-index",
    endpoint="https://app.phoenix.arize.com/v1/traces"
)

In [43]:
llama_index.core.set_global_handler(tracer_provider)

In [42]:
import llama_index
import os

PHOENIX_API_KEY = 'fd4702dfe89bc396e40:491f157'
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix",
    endpoint="https://llamatrace.com/v1/traces"
)
